In [118]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

data_folder_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, "data_folder"))
sys.path.append(data_folder_path) 

from synth_dim_model import *
from paraparticle_utils import *

import numpy as np
import scipy
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import concurrent.futures
plt.rcParams.update({'font.size': 12})

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
np.set_printoptions(precision=5, suppress=True, linewidth=100)


<Token var=<ContextVar name='format_options' default={'edgeitems': 3, 'threshold': 1000, 'floatmode': 'maxprec', 'precision': 8, 'suppress': False, 'linewidth': 75, 'nanstr': 'nan', 'infstr': 'inf', 'sign': '-', 'formatter': None, 'legacy': 9223372036854775807, 'override_repr': None} at 0x0000022D1D556ED0> at 0x0000022D3577BC80>

In [119]:
N = 2

H = construct_hamiltonian(N, M = 3, V = 1, mu = 0, J = 0)
eigenvalues, eigenvectors = exact_diagonalize(H)

ground_state_manifold = construct_ground_state_manifold(eigenvalues, eigenvectors)

states, formatted_states = enumerate_states(N, M = 3)

In [120]:
def build_spin_operator(N, site_index, alpha, sign):
    """
    Build the operator x_{i,alpha}^{(sign)} as a 3^N x 3^N matrix,
    acting on site_index ∈ [0, N-1].
    
    Args:
        N (int): number of sites
        site_index (int): site to act on (0-indexed)
        alpha (int): 1 or 2, selects which x^{(±)}_{i,alpha}
        sign (str): '+' or '-'
    
    Returns:
        np.ndarray: 3^N x 3^N matrix representing x_{i,alpha}^{(sign)}
    """
    assert 0 <= site_index < N
    assert alpha in [1, 2]
    assert sign in ['+', '-']

    id3 = np.eye(3)

    x_matrices = {
        ('+', 1): np.array([[0, 0, 0],
                            [0, 0, 0],
                            [0, 1, 0]]),
        ('-', 1): np.array([[0, 0, 0],
                            [0, 0, 1],
                            [0, 0, 0]]),
        ('+', 2): np.array([[0, 1, 0],
                            [0, 0, 0],
                            [0, 0, 0]]),
        ('-', 2): np.array([[0, 0, 0],
                            [1, 0, 0],
                            [0, 0, 0]]),
    }

    op_list = [id3] * site_index + [x_matrices[(sign, alpha)]] + [id3] * (N - site_index - 1)
    full_op = op_list[0]
    for op in op_list[1:]:
        full_op = np.kron(full_op, op)

    return full_op

In [121]:
state = eigenvectors[0]

for i, value in enumerate(state):
    print(f"{value}: {formatted_states[i]}")
    


0j: |0,0>
(0.7071067811865477+0j): |0,1>
0j: |0,2>
(-0.7071067811865475+0j): |1,0>
0j: |1,1>
0j: |1,2>
0j: |2,0>
0j: |2,1>
0j: |2,2>


In [122]:
new_state = x_1 @ state

if np.linalg.norm(new_state) != 0:
    new_state = new_state/np.linalg.norm(new_state)

In [123]:
for i, value in enumerate(new_state):
    print(f"{value}: {formatted_states[i]}")

0j: |0,0>
0j: |0,1>
0j: |0,2>
0j: |1,0>
0j: |1,1>
0j: |1,2>
(-1+0j): |2,0>
0j: |2,1>
0j: |2,2>
